In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import classification_report


from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB 


import warnings
warnings.filterwarnings('ignore')


In [50]:
train_df= pd.read_csv('train_obesity.csv')
test_df= pd.read_csv('test_obesity.csv')

### Explore Data

In [51]:
train_df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II


In [52]:
train_df.sample()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
15639,15639,Male,18.0,1.85,65.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,1.0,Sometimes,Public_Transportation,Normal_Weight


In [53]:
train_df.tail()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
20753,20753,Male,25.137087,1.766626,114.187096,yes,yes,2.919584,3.000000,Sometimes,no,2.151809,no,1.330519,0.196680,Sometimes,Public_Transportation,Obesity_Type_II
20754,20754,Male,18.000000,1.710000,50.000000,no,yes,3.000000,4.000000,Frequently,no,1.000000,no,2.000000,1.000000,Sometimes,Public_Transportation,Insufficient_Weight
20755,20755,Male,20.101026,1.819557,105.580491,yes,yes,2.407817,3.000000,Sometimes,no,2.000000,no,1.158040,1.198439,no,Public_Transportation,Obesity_Type_II
20756,20756,Male,33.852953,1.700000,83.520113,yes,yes,2.671238,1.971472,Sometimes,no,2.144838,no,0.000000,0.973834,no,Automobile,Overweight_Level_II
20757,20757,Male,26.680376,1.816547,118.134898,yes,yes,3.000000,3.000000,Sometimes,no,2.003563,no,0.684487,0.713823,Sometimes,Public_Transportation,Obesity_Type_II


In [54]:
train_df.shape

(20758, 18)

In [55]:
train_df.columns

Index(['id', 'Gender', 'Age', 'Height', 'Weight',
       'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC',
       'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')

In [56]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   Gender                          20758 non-null  object 
 2   Age                             20758 non-null  float64
 3   Height                          20758 non-null  float64
 4   Weight                          20758 non-null  float64
 5   family_history_with_overweight  20758 non-null  object 
 6   FAVC                            20758 non-null  object 
 7   FCVC                            20758 non-null  float64
 8   NCP                             20758 non-null  float64
 9   CAEC                            20758 non-null  object 
 10  SMOKE                           20758 non-null  object 
 11  CH2O                            20758 non-null  float64
 12  SCC                             

In [57]:
train_df.describe()

,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,20758.00000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000,20758.000000
mean,10378.50000,23.841804,1.700245,87.887768,2.445908,2.761332,2.029418,0.981747,0.616756
std,5992.46278,5.688072,0.087312,26.379443,0.533218,0.705375,0.608467,0.838302,0.602113
min,0.00000,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,5189.25000,20.000000,1.631856,66.000000,2.000000,3.000000,1.792022,0.008013,0.000000
50%,10378.50000,22.815416,1.700000,84.064875,2.393837,3.000000,2.000000,1.000000,0.573887
75%,15567.75000,26.000000,1.762887,111.600553,3.000000,3.000000,2.549617,1.587406,1.000000
max,20757.00000,61.000000,1.975663,165.057269,3.000000,4.000000,3.000000,3.000000,2.000000


In [58]:
train_df.isnull().sum()

id                                0
Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [59]:
train_df.duplicated().sum()

0

### Transform Data

In [60]:
label = LabelEncoder()

In [61]:
obj = train_df.select_dtypes(include='object')
non_obj = train_df.select_dtypes(exclude='object')

In [62]:
for col in range(0,obj.shape[1]):
    obj.iloc[:,col]=label.fit_transform(obj.iloc[:,col])
obj= obj.astype(int)

In [63]:
df=pd.concat([non_obj,obj],axis=1)

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   Age                             20758 non-null  float64
 2   Height                          20758 non-null  float64
 3   Weight                          20758 non-null  float64
 4   FCVC                            20758 non-null  float64
 5   NCP                             20758 non-null  float64
 6   CH2O                            20758 non-null  float64
 7   FAF                             20758 non-null  float64
 8   TUE                             20758 non-null  float64
 9   Gender                          20758 non-null  int32  
 10  family_history_with_overweight  20758 non-null  int32  
 11  FAVC                            20758 non-null  int32  
 12  CAEC                            

In [65]:
test_df.head()

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


In [66]:
# for test
objj = test_df.select_dtypes(include='object')
non_objj = test_df.select_dtypes(exclude='object')

In [67]:
# for test
for col in range(0,objj.shape[1]):
    objj.iloc[:,col]=label.fit_transform(objj.iloc[:,col])
objj= objj.astype(int)

In [68]:
df2= pd.concat([non_objj,objj],axis=1)

In [69]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13840 entries, 0 to 13839
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              13840 non-null  int64  
 1   Age                             13840 non-null  float64
 2   Height                          13840 non-null  float64
 3   Weight                          13840 non-null  float64
 4   FCVC                            13840 non-null  float64
 5   NCP                             13840 non-null  float64
 6   CH2O                            13840 non-null  float64
 7   FAF                             13840 non-null  float64
 8   TUE                             13840 non-null  float64
 9   Gender                          13840 non-null  int32  
 10  family_history_with_overweight  13840 non-null  int32  
 11  FAVC                            13840 non-null  int32  
 12  CAEC                            

In [70]:
scaler=StandardScaler()

In [71]:
# for train
sc1=df[['Age']]
sc2=df[['Weight']]
df['Age']=scaler.fit_transform(sc1)
df['Weight']=scaler.fit_transform(sc2)

In [72]:
df.head()

,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS,NObeyesdad
0,0,0.105699,1.699998,-0.235713,2.000000,2.983297,2.763573,0.000000,0.976473,1,1,1,2,0,0,1,3,6
1,1,-1.027052,1.560000,-1.170931,2.000000,3.000000,2.000000,1.000000,1.000000,0,1,1,1,0,0,2,0,1
2,2,-1.027052,1.711460,-1.430012,1.880534,1.411685,1.910378,0.866045,1.673584,0,1,1,2,0,0,2,3,0
3,3,-0.507929,1.710730,1.644770,3.000000,3.000000,1.674061,1.467863,0.780199,0,1,1,2,0,0,1,3,4
4,4,1.371197,1.914186,0.224054,2.679664,1.971472,1.979848,1.967973,0.931721,1,1,1,2,0,0,1,3,6


In [73]:
# for test 
sc1=df2[['Age']]
sc2=df2[['Weight']]
df2['Age']=scaler.fit_transform(sc1)
df2['Weight']=scaler.fit_transform(sc2)

In [74]:
df2.head()

,id,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender,family_history_with_overweight,FAVC,CAEC,SMOKE,SCC,CALC,MTRANS
0,20758,0.508163,1.848294,1.273786,2.938616,3.000000,2.825629,0.855400,0.000000,1,1,1,2,0,0,2,3
1,20759,-0.509128,1.600000,-0.818988,2.000000,1.000000,3.000000,1.000000,0.000000,0,1,1,2,0,0,2,3
2,20760,0.353000,1.643355,0.927432,3.000000,3.000000,2.621877,0.000000,0.250502,0,1,1,2,0,0,2,3
3,20761,-0.512705,1.553127,0.623672,2.000000,2.977909,2.786417,0.094851,0.000000,1,1,1,2,0,0,2,3
4,20762,0.353000,1.627396,0.668336,3.000000,3.000000,2.653531,0.000000,0.741069,0,1,1,2,0,0,2,3


### Creat model

In [75]:
x=df.drop(['NObeyesdad','id'],axis=1)
y=df['NObeyesdad']

In [76]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1234)

In [77]:
m1=LogisticRegression()
m2=SVC()
m3=RandomForestClassifier()
m4=GradientBoostingClassifier()
m5=KNeighborsClassifier()
m6=XGBClassifier()
m7=GaussianNB()

In [78]:
def class_report(model):
    model.fit(x_train,y_train)
    pred = model.predict(x_test)
    print(classification_report(pred,y_test))

In [79]:
class_report(m1)

              precision    recall  f1-score   support

           0       0.87      0.80      0.83       555
           1       0.70      0.71      0.70       569
           2       0.82      0.78      0.80       616
           3       0.96      0.93      0.95       672
           4       1.00      1.00      1.00       830
           5       0.61      0.68      0.64       439
           6       0.60      0.65      0.62       471

    accuracy                           0.81      4152
   macro avg       0.79      0.79      0.79      4152
weighted avg       0.82      0.81      0.82      4152



In [80]:
class_report(m2)

              precision    recall  f1-score   support

           0       0.95      0.87      0.91       560
           1       0.78      0.83      0.80       545
           2       0.84      0.81      0.82       611
           3       0.97      0.95      0.96       661
           4       0.99      1.00      1.00       828
           5       0.66      0.72      0.69       446
           6       0.69      0.69      0.69       501

    accuracy                           0.86      4152
   macro avg       0.84      0.84      0.84      4152
weighted avg       0.86      0.86      0.86      4152



In [81]:
class_report(m3)

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       498
           1       0.91      0.85      0.88       613
           2       0.89      0.90      0.89       583
           3       0.97      0.97      0.97       651
           4       0.99      1.00      1.00       827
           5       0.74      0.80      0.77       454
           6       0.82      0.79      0.80       526

    accuracy                           0.90      4152
   macro avg       0.89      0.89      0.89      4152
weighted avg       0.90      0.90      0.90      4152



In [82]:
class_report(m4)

              precision    recall  f1-score   support

           0       0.94      0.95      0.94       501
           1       0.91      0.86      0.88       615
           2       0.91      0.91      0.91       585
           3       0.98      0.97      0.97       652
           4       0.99      1.00      1.00       826
           5       0.75      0.81      0.78       455
           6       0.83      0.81      0.82       518

    accuracy                           0.91      4152
   macro avg       0.90      0.90      0.90      4152
weighted avg       0.91      0.91      0.91      4152



In [83]:
class_report(m5)

              precision    recall  f1-score   support

           0       0.85      0.77      0.81       565
           1       0.66      0.62      0.64       616
           2       0.81      0.72      0.76       664
           3       0.96      0.94      0.95       661
           4       0.99      1.00      0.99       827
           5       0.46      0.58      0.51       396
           6       0.56      0.66      0.61       423

    accuracy                           0.78      4152
   macro avg       0.76      0.75      0.75      4152
weighted avg       0.79      0.78      0.79      4152



In [84]:
class_report(m6)

              precision    recall  f1-score   support

           0       0.92      0.95      0.94       495
           1       0.91      0.87      0.89       605
           2       0.90      0.90      0.90       589
           3       0.97      0.97      0.97       651
           4       0.99      1.00      1.00       826
           5       0.79      0.82      0.80       475
           6       0.83      0.82      0.82       511

    accuracy                           0.91      4152
   macro avg       0.90      0.90      0.90      4152
weighted avg       0.91      0.91      0.91      4152



In [85]:
class_report(m7)

              precision    recall  f1-score   support

           0       0.84      0.69      0.76       621
           1       0.46      0.66      0.54       402
           2       0.65      0.40      0.49       962
           3       0.94      0.73      0.82       836
           4       0.99      0.95      0.97       864
           5       0.27      0.59      0.37       224
           6       0.25      0.53      0.34       243

    accuracy                           0.67      4152
   macro avg       0.63      0.65      0.61      4152
weighted avg       0.74      0.67      0.69      4152



## Get the best params for increasing accuracy

### m3=RandomForestClassifier()
### m4=GradientBoostingClassifier()
### m6=XGBClassifier()

In [86]:
params_grid={
           'n_estimators':[100,200,300],
           'learning_rate':[0.1,0.01,0.001],
           'max_depth' : [3,5,7]
        }

scorer='accuracy'

In [87]:
model3=GridSearchCV(m3, params_grid, scoring=scorer,n_jobs=-1)
model3.fit(x_train,y_train)
print(model3.best_params_)
print(model3.best_score_)

'\nmodel3=GridSearchCV(m3, params_grid, scoring=scorer,n_jobs=-1)\nmodel3.fit(x_train,y_train)\nprint(model3.best_params_)\nprint(model3.best_score_)\n'

In [ ]:
model4=GridSearchCV(m4, params_grid, scoring=scorer,n_jobs=-1)
model4.fit(x_train,y_train)
print(model4.best_params_)
print(model4.best_score_)

In [ ]:
model6=GridSearchCV(m6, params_grid, scoring=scorer,n_jobs=-1)
model6.fit(x_train,y_train)
print(model6.best_params_)
print(model6.best_score_)

For test data

In [89]:
testx=df2.drop('id',axis=1)

In [90]:
pred = m6.predict(testx)

In [94]:
resule_test=pd.DataFrame({'id':df2['id'],
                          'NObeyesdad':pred})

In [96]:
#resule_test['NObeyesdad']=label.inverse_transform(pred)

In [97]:
resule_test

,id,NObeyesdad
0,20758,3
1,20759,5
2,20760,4
3,20761,2
4,20762,4
...,...,...
13835,34593,6
13836,34594,5
13837,34595,0
13838,34596,1


In [ ]:
resule_test.to_csv('submission.csv',index=False)